<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

In [1]:
# !pip3 install minio
# !pip3 install pandas
# !sudo apt-get install -y mosquitto-clients

In [2]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

#df_metrics = pd.DataFrame(columns=['Workload_size','Workload TPT','Average Workload TPT','Average FET','Average CT'])
http://172.17.141.197:3000/goto/MU1mA-MVk?orgId=1

In [3]:
#scenario="emqx_testing_namespace_scaling_without_max_inflight_replicas_2_10_3"
scenario = "resource_usage"
user=1
iteration=1
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:31022/'


In [4]:
!rm input_data.csv
!touch input_data.csv
!echo "intime" > input_data.csv


In [5]:
#!cp /home/ubuntu/scaling-sdp/aeneas/dataset/p001.mp3 .

In [6]:
!locust --headless -f mqtt_locust.py --host='127.0.0.1' -t 1  --users 1 --spawn-rate 2

[2022-12-02 11:01:14,928] ubuntu/INFO/locust.main: Run time limit set to 1 seconds
[2022-12-02 11:01:14,929] ubuntu/INFO/locust.main: Starting Locust 2.12.0
Type     Name  # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
--------||-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated       0     0(0.00%) |      0       0       0      0 |    0.00        0.00

[2022-12-02 11:01:14,931] ubuntu/INFO/locust.runners: Ramping to 1 users at a rate of 2.00 per second
[2022-12-02 11:01:14,933] ubuntu/INFO/locust.runners: All users spawned: {"MQTTLocust": 1} (1 total users)
Disconnected result code None
Disconnected result code None
[2022-12-02 11:01:15,228] ubuntu/INFO/locust.main: --run-time limit reached, shutting down
[2022-12-02 11:01:15,231] ubuntu/INFO/locust.main: Shutting down (exit code 0)
Type     Name  # reqs      # fails |    Avg     M

In [7]:
#cmd = "./mqtt_client.sh "+str(user)
#with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
#    output1 = process.communicate()[0].decode("utf-8")
time.sleep(40)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:

df= pd.read_csv('input_data.csv')
print(df)
df_metrics=pd.read_csv('data.csv')

                         intime
0  '2022-12-02 11:01:14.953585'


In [9]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)#
    client.remove_object(bucket_name, obj.object_name)


In [10]:
df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime']).dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'])
df =df.sort_values(by="outtime")
display(df)
print(df['outtime'].iloc[-1]-df['intime'].iloc[0])

,intime,outtime
0,2022-12-02 11:01:14.953585,2022-12-02 11:01:21.604


0 days 00:00:06.650415


In [11]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))    


In [12]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-xlhm8  -n openfaas -c gateway  | grep /function/aeneas | cut -c 81-85 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-xlhm8 -n openfaas -c gateway  | grep /function/scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]
df

,intime,outtime,aeneas,tocloud
0,2022-12-02 11:01:14.953585,2022-12-02 11:01:21.604,4.068,0.0452


In [13]:
#print(df['aeneas'])
#print(df['tocloud'])
#TPT: Total Processing Time
#FET: Function Execution Time
#CT: Communication Time
df =df.sort_values(by="outtime")
df['TPT']=(df['outtime']-df['intime']).dt.seconds
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
df['Waiting time']= (df['TPT']).astype(float) - df['FET']

display(df)
df["scenario"]= ""+str(user)+"_"+str(iteration)+"_"+scenario


,intime,outtime,aeneas,tocloud,TPT,FET,Waiting time
0,2022-12-02 11:01:14.953585,2022-12-02 11:01:21.604,4.068,0.0452,6,4.1132,1.8868


In [14]:
pre_url = PROMETHEUS + '/api/v1/query?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time = (time.mktime(timestamp.timetuple()))
#interval= int(df['TPT'].sum()) if int(df['TPT'].sum()) > 60  else 60


In [15]:
interval = (df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()
interval= round(interval)

In [16]:
interval= round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) if round((df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds()) > 60  else 60
print(interval)

60


In [17]:
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    data=res.get('data').get('result')[0].get('value')[1]
    return data

In [18]:
#Pod Stats
expr_pod_cpu_aeneas_function = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"aeneas.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_tocloud_function = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_broker = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"emqx.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_connector_aeneas = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"aeneas.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_connector_tocloud = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"tocloud.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_functions_all = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time)

df['pod_cpu_aeneas_function']=getdataprometheus(pre_url+expr_pod_cpu_aeneas_function)
df['pod_cpu_tocloud_function']=getdataprometheus(pre_url+expr_pod_cpu_tocloud_function)
df['pod_cpu_mosquitto_broker']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_broker)
df['pod_cpu_mosquitto_connector_aeneas']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_aeneas)
df['pod_cpu_mosquitto_connector_tocloud']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_tocloud)
df['pod_cpu_functions_all']=getdataprometheus(pre_url+expr_pod_cpu_functions_all)


In [19]:
df_metrics = df_metrics.append(df, ignore_index=True)
df_metrics.to_csv("data.csv",index=False)
display(df_metrics)

/tmp/ipykernel_23146/3101442244.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metrics = df_metrics.append(df, ignore_index=True)


,scenario,intime,outtime,aeneas,tocloud,TPT,FET,Waiting time,pod_cpu_aeneas_function,pod_cpu_tocloud_function,pod_cpu_mosquitto_broker,pod_cpu_mosquitto_connector_aeneas,pod_cpu_mosquitto_connector_tocloud,pod_cpu_functions_all
0,1_1_resource_usage,2022-12-01 18:49:56.380118,2022-12-01 18:50:02.954,4.118,0.0415,6,4.1595,1.8405,0.003561,0.00402,0.034577,0.024839,0.023136,0.01167
1,1_1_resource_usage,2022-12-01 18:52:16.525348,2022-12-01 18:52:22.057000,4.028,0.0384,5,4.0664,0.9336,0.108619,0.00373,0.028371,0.02798,0.0,0.114632
2,1_1_resource_usage,2022-12-01 19:20:09.493529,2022-12-01 19:20:15.414000,4.055,0.0425,5,4.0975,0.9025,0.003992,0.004015,0.027063,0.136824,0.019329,0.013349
3,1_1_resource_usage,2022-12-01 19:25:32.694148,2022-12-01 19:25:38.233000,4.026,0.0398,5,4.0658,0.9342,0.004399,0.003966,0.025186,0.026365,0.025079,0.013947
4,1_1_resource_usage,2022-12-01 19:33:38.592264,2022-12-01 19:33:44.490000,4.031,0.0379,5,4.0689,0.9311,0.003717,0.004468,0.029632,0.012512,0.019951,0.013355
5,1_1_resource_usage,2022-12-01 19:42:06.285642,2022-12-01 19:42:11.476000,4.033,0.0414,5,4.0744,0.9256,0.004231,0.003788,0.035133,0.011785,0.012181,0.010219
6,1_1_resource_usage,2022-12-02 09:59:09.379479,2022-12-02 09:59:15.458000,4.057,0.038,6,4.0950,1.9050,0.003516,0.004177,0.038106,0.010921,0.007276,0.011493
7,1_1_resource_usage,2022-12-02 10:10:56.315323,2022-12-02 10:11:02.740000,4.034,0.0408,6,4.0748,1.9252,0.016082,0.002789,0.033197,0.025091,0.024858,0.023914
8,1_1_resource_usage,2022-12-02 10:20:48.306775,2022-12-02 10:20:53.518000,4.064,0.0396,5,4.1036,0.8964,0.029344,0.004756,0.0,0.025746,0.018616,0.039305
9,1_1_resource_usage,2022-12-02 11:01:14.953585,2022-12-02 11:01:21.604000,4.068,0.0452,6,4.1132,1.8868,0.02715786126816026,0.0029489358908481185,0.029098900729729416,0.168291239608318,0.01392317378115719,0.031041526359009433


In [20]:
# user=df.shape[0]
# metrics = {'workload_type':str(user)+"_"+scenario,'workload_size':user,'Workload TPT':(df.loc[user-1, 'outtime'] - df.loc[0, 'intime']).total_seconds(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean(),'Average Waiting Time':df['Waiting time'].mean()}


In [21]:
# expr_cpu='sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace=~"openfaas.*|default"}[5m]))&time='+str(time)
# expr_memory='sum(container_memory_working_set_bytes{container_name!="POD",namespace=~"openfaas.*|default"})&time='+str(time)
# expr_disk_read ='sum(rate(container_fs_reads_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
# expr_disk_write='sum(rate(container_fs_writes_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
# expr_network_transmit='sum(rate(container_network_transmit_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
# expr_network_recieve='sum(rate(container_network_receive_bytes_total{namespace=~"openfaas.*|default"}['+str(interval)+'s]))&time='+str(time)
# #expr_cpu = '100 - (avg (irate(node_cpu_seconds_total{mode="idle"}['+str(interval)+'s])) * 100)&time='+str(time)
# #expr_memory = '100 - ((sum(node_memory_MemAvailable_bytes) / sum(node_memory_MemTotal_bytes))*100)&time='+str(time)
# #expr_disk_read = 'sum(rate(node_disk_read_bytes_total['+str(interval)+'s]))&time='+str(time)
# #expr_disk_write =  'sum(rate(node_disk_written_bytes_total['+str(interval)+'s]))&time='+str(time)
# #expr_network_transmit= 'sum(rate(node_network_transmit_errs_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
# #expr_network_recieve='sum(rate(node_network_receive_packets_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
# metrics['C_CPU'] = getdataprometheus(pre_url+expr_cpu)
# metrics['C_RAM'] = getdataprometheus(pre_url+expr_memory)
# metrics['C_disk_read']=getdataprometheus(pre_url+expr_disk_read)
# metrics['C_disk_write']=getdataprometheus(pre_url+expr_disk_write)
# metrics['C_network_recieve']=getdataprometheus(pre_url+expr_network_transmit)
# metrics['C_network_transmit']=getdataprometheus(pre_url+expr_network_recieve)

In [22]:
# #Pod Stats
# expr_pod_cpu_aeneas_function = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"aeneas.*"}['+str(interval)+'s]))&time='+str(time)
# expr_pod_cpu_tocloud_function = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling.*"}['+str(interval)+'s]))&time='+str(time)
# expr_pod_cpu_mosquitto_broker = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"emqx.*"}['+str(interval)+'s]))&time='+str(time)
# expr_pod_cpu_mosquitto_connector_aeneas = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"aeneas.*"}['+str(interval)+'s]))&time='+str(time)
# expr_pod_cpu_mosquitto_connector_tocloud = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"tocloud.*"}['+str(interval)+'s]))&time='+str(time)
# expr_pod_cpu_functions_all = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time)

# metrics['pod_cpu_aeneas_function']=getdataprometheus(pre_url+expr_pod_cpu_aeneas_function)
# metrics['pod_cpu_tocloud_function']=getdataprometheus(pre_url+expr_pod_cpu_tocloud_function)
# metrics['pod_cpu_mosquitto_broker']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_broker)
# metrics['pod_cpu_mosquitto_connector_aeneas']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_aeneas)
# metrics['pod_cpu_mosquitto_connector_tocloud']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_tocloud)
# metrics['pod_cpu_functions_all']=getdataprometheus(pre_url+expr_pod_cpu_functions_all)



In [23]:
#!echo "workload_type,workload_size,Workload TPT,Average Workload TPT,Average FET,Average Waiting Time,C_CPU,C_RAM,C_disk_read,C_disk_write,C_network_recieve,C_network_transmit,pod_cpu_aeneas_function,pod_cpu_functions_all,pod_cpu_mosquitto_broker,pod_cpu_mosquitto_connector_aeneas,pod_cpu_mosquitto_connector_tocloud" > metrics.csv

In [24]:
# df_metrics = pd.read_csv("metrics.csv")
# df_metrics = df_metrics.append(metrics, ignore_index=True)


In [25]:
# df_metrics.to_csv("metrics.csv",index=False)
# #df_metrics=df_metrics.sort_values(by="workload_size")
# display(df_metrics)

In [26]:
# pod_metrics = {}
# expr_pod_cpu_aeneas = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}['+str(interval)+'s]))&time='+str(time)
# pod_metrics['cpu_aeneas'] = getdataprometheus(pre_url+expr_pod_cpu_aeneas)
# expr_pod_memory_aeneas = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}['+str(interval)+'s]))&time='+str(time)
# pod_metrics['memory_aeneas'] = getdataprometheus(pre_url+expr_pod_memory_aeneas)
# expr_pod_cpu_tocloud = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-tocloud.*"}['+str(interval)+'s]))&time='+str(time)
# pod_metrics['cpu_tocloud'] = getdataprometheus(pre_url+expr_pod_cpu_aeneas)
# expr_pod_memory_tocloud = 'sum(rate(container_memory_working_set_bytes{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-tolcoud.*"}['+str(interval)+'s]))&time='+str(time)
# pod_metrics['memory_tocloud'] = getdataprometheus(pre_url+expr_pod_memory_aeneas)


In [27]:
# #pod_metrics = json.loads(pod_metrics)
# df2 = pd.DataFrame.from_dict(pod_metrics, orient="index")
# print(df2)